In [1]:
import sys
sys.path.insert(0, "../../../")

In [2]:
import deeplay as dl

mnist_train = dl.datasets.MNIST("../../data", download=True, train=True, backend="torch")
mnist_test = dl.datasets.MNIST("../../data", download=True, train=False, backend="torch")



In [3]:
train_dataloader = dl.DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloader = dl.DataLoader(mnist_test, batch_size=32, shuffle=False)

In [4]:
# OPTION 1
mlp = dl.MultiLayerPerceptron(28*28, [32, 32], 10)

# OPTION 2: 
mlp_config = (
    dl.Config()
    .in_features(28*28)
    .hidden_dims([32, 32])
    .out_features(10)
)

mlp = dl.MultiLayerPerceptron.from_config(mlp_config)
print(mlp)

MultiLayerPerceptron(
  (in_features): 784
  (hidden_dims): [32, 32]
  (out_features): 10
  (depth): 3
  (blocks): ModuleList(
    (0): Template(
      (layer): Linear(in_features=784, out_features=32, bias=True)
      (normalization): Identity()
      (activation): ReLU()
      (dropout): Identity()
    )
    (1): Template(
      (layer): Linear(in_features=32, out_features=32, bias=True)
      (normalization): Identity()
      (activation): ReLU()
      (dropout): Identity()
    )
  )
  (out_layer): Linear(in_features=32, out_features=10, bias=True)
  (out_activation): Identity()
)


In [6]:
# OPTION 1
classifier = dl.Classifier(mlp)

# OPTION 2
classifier_config = (
    dl.Config()
    .model(dl.MultiLayerPerceptron, mlp_config)
)
classifier = dl.Classifier.from_config(classifier_config)

# OPTION 3
classifier_config = (
    dl.Config()
    .model(dl.MultiLayerPerceptron, in_features=28*28, hidden_dims=[32, 32], out_features=10)
)
classifier = dl.Classifier.from_config(classifier_config)

# OPTION 4
class Classifier(dl.Application):

    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        
        # Compute loss
        # very hard without torch. 
        loss = ...

        return loss    

In [ ]:
mlp = dl.MultiLayerPerceptron(28*28, [32, 32], 10)
classifier = Classifier(mlp)
trainer = dl.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(classifier, train_dataloader)

In [ ]:
trainer.test(classifier, test_dataloader)